reading the files

In [1]:
import pandas as pd 

In [2]:
manufacturer_df = pd.read_pickle('cumulative_manufacturer.pkl')
manufacturer_df.head()

,manufacture_id,year,month,region,city_lat,city_long,city,state,country,quantity
0,1,2021,1,West,33.1447,117.1658,San Marcos,CA,US,75
1,2,2020,6,Unknown,0.0000,0.0000,Unknown,Unknown,Unknown,12
2,3,2021,7,Unknown,37.5665,126.9780,Seoul,Unknown,South Korea,4
3,4,2018,11,West,37.4419,122.1430,Palo Alto,CA,US,11
4,5,2019,5,West,33.1959,117.3795,Oceanside,CA,US,1


In [4]:
ev_car_df = pd.read_pickle('cumulative_df.pkl')
ev_car_df.head()

,year,month,region,city_lat,city_long,city,state,country,vehicle_pop_quantity,ev_cs_quantity
0,0,0,Midwest,29.093133,-97.315846,Dewitt,MI,US,0.0,1.0
1,0,0,Midwest,31.997366,-102.077948,Midland,MI,US,0.0,5.0
2,0,0,Midwest,34.341053,-82.950328,Hart,MI,US,0.0,1.0
3,0,0,Midwest,38.577359,-92.172427,Jefferson City,MO,US,0.0,2.0
4,0,0,Midwest,38.647738,-88.923211,Marion,IL,US,0.0,1.0


creating a new data frame 

In [11]:
cumulative_df = pd.merge(ev_car_df,manufacturer_df, how='outer', on= ['year','month',	'region'	,'city_lat','city_long','city',	'state'	,'country'])

In [12]:
cumulative_df.head(5)

,year,month,region,city_lat,city_long,city,state,country,vehicle_pop_quantity,ev_cs_quantity,manufacture_id,quantity
0,0,0,Midwest,29.093133,-97.315846,Dewitt,MI,US,0.0,1.0,NaN,NaN
1,0,0,Midwest,31.997366,-102.077948,Midland,MI,US,0.0,5.0,NaN,NaN
2,0,0,Midwest,34.341053,-82.950328,Hart,MI,US,0.0,1.0,NaN,NaN
3,0,0,Midwest,38.577359,-92.172427,Jefferson City,MO,US,0.0,2.0,NaN,NaN
4,0,0,Midwest,38.647738,-88.923211,Marion,IL,US,0.0,1.0,NaN,NaN


In [16]:
cumulative_df['vehicle_pop_quantity'] = cumulative_df['vehicle_pop_quantity'].fillna(0).astype(int)
cumulative_df['manufacture_id'] = cumulative_df['manufacture_id'].fillna(0).astype(int)
cumulative_df['quantity'] = cumulative_df['quantity'].fillna(0).astype(int)

In [17]:
cumulative_df.head(5)

,year,month,region,city_lat,city_long,city,state,country,vehicle_pop_quantity,ev_cs_quantity,manufacture_id,quantity
0,0,0,Midwest,29.093133,-97.315846,Dewitt,MI,US,0,1.0,0,0
1,0,0,Midwest,31.997366,-102.077948,Midland,MI,US,0,5.0,0,0
2,0,0,Midwest,34.341053,-82.950328,Hart,MI,US,0,1.0,0,0
3,0,0,Midwest,38.577359,-92.172427,Jefferson City,MO,US,0,2.0,0,0
4,0,0,Midwest,38.647738,-88.923211,Marion,IL,US,0,1.0,0,0


spliting into the tables

we need to add only manufacturer 0 as a null value 

In [18]:
manufacturer_dim = cumulative_df[['manufacture_id']].drop_duplicates(keep='first')
manufacturer_dim

,manufacture_id
0,0
18582,1
18583,2
18584,3
18585,4
18586,5
18587,6
18588,7
18589,8
18590,9


we can try to see if we can get location id from the manufacturer table and create only new location id for ev charging facts
first check to see if adding location id will cause increase in the df

date dim 

In [19]:
cumulative_df.columns

Index(['year', 'month', 'region', 'city_lat', 'city_long', 'city', 'state',
       'country', 'vehicle_pop_quantity', 'ev_cs_quantity', 'manufacture_id',
       'quantity'],
      dtype='object')

In [21]:
month_dim = cumulative_df[['month','year']].drop_duplicates(keep='first')
month_dim

,month,year
0,0,0
514,8,1995
515,10,1996
516,1,1997
517,7,1997
...,...,...
18394,1,2023
18594,2,2023
18597,8,2022
18601,6,2022


loading the tables 